##### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Approximate Nearest Neighbor(ANN) 및 텍스트 임베딩을 사용한 의미론적 검색


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/tf2_semantic_approximate_nearest_neighbors"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/hub/tutorials/tf2_semantic_approximate_nearest_neighbors.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/hub/tutorials/tf2_semantic_approximate_nearest_neighbors.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 보기</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/hub/tutorials/tf2_semantic_approximate_nearest_neighbors.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운론드하기</a></td>
  <td><a href="https://tfhub.dev/google/nnlm-en-dim128/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF 허브 모델 보기</a></td>
</table>

이 튜토리얼에서는 입력 데이터가 제공된 [TensorFlow Hub](https://tfhub.dev)(TF-Hub) 모듈에서 임베딩을 생성하고 추출된 임베딩을 사용하여 approximate nearest neighbour(ANN) 인덱스를 빌드하는 방법을 보여줍니다. 그런 다음 이 인덱스를 실시간 유사성 일치 및 검색에 사용할 수 있습니다.

많은 양의 데이터를 처리할 때 전체 리포지토리를 스캔하여 주어진 쿼리와 가장 유사한 항목을 실시간으로 찾는 식으로 정확한 일치 작업을 수행하는 것은 효율적이지 않습니다. 따라서 속도를 크게 높이기 위해 정확한 nearest neighbor(NN) 일치를 찾을 때 약간의 정확성을 절충할 수 있는 근사 유사성 일치 알고리즘을 사용합니다.

이 튜토리얼에서는 쿼리와 가장 유사한 헤드라인을 찾기 위해 뉴스 헤드라인 자료의 텍스트를 실시간으로 검색하는 예를 보여줍니다. 키워드 검색과 달리 이 검색으로 텍스트 임베딩에 인코딩된 의미론적 유사성이 포착됩니다.

이 튜토리얼의 단계는 다음과 같습니다.

1. 샘플 데이터를 다운로드합니다.
2. TF-Hub 모델을 사용하여 데이터에 대한 임베딩을 생성합니다.
3. 임베딩의 ANN 인덱스를 빌드합니다.
4. 유사성 일치에 인덱스를 사용합니다.

[Apache Beam](https://beam.apache.org/documentation/programming-guide/)을 사용하여 TF-Hub 모델에서 임베딩을 생성합니다. 또한 Spotify의 [ANNOY](https://github.com/spotify/annoy) 라이브러리를 사용하여 근사적으로 최근접 인덱스를 빌드합니다.

### 더 많은 모델

아키텍처는 동일하지만 다른 언어로 훈련된 모델의 경우, [이](https://tfhub.dev/google/collections/nnlm/1) 컬렉션을 참조하세요. [여기](https://tfhub.dev/s?module-type=text-embedding)에서 현재 [tfhub.dev](https://tfhub.dev/)에서 호스팅되는 모든 텍스트 임베딩을 확인할 수 있습니다.

## 설정

필요한 라이브러리를 설치합니다.

In [1]:
!pip install apache_beam
!pip install 'scikit_learn~=0.23.0'  # For gaussian_random_matrix.
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 19.5 MB/s eta 0:00:00
  Created wheel for crcmod: filename=crcmod

필요한 라이브러리를 가져옵니다.

In [2]:
import os
import sys
import pickle
from collections import namedtuple
from datetime import datetime
import numpy as np
import apache_beam as beam
from apache_beam.transforms import util
import tensorflow as tf
import tensorflow_hub as hub
import annoy
from sklearn.random_projection import gaussian_random_matrix

ImportError: cannot import name 'gaussian_random_matrix' from 'sklearn.random_projection' (/usr/local/lib/python3.10/dist-packages/sklearn/random_projection.py)

In [3]:
print('TF version: {}'.format(tf.__version__))
print('TF-Hub version: {}'.format(hub.__version__))
print('Apache Beam version: {}'.format(beam.__version__))

TF version: 2.17.0
TF-Hub version: 0.16.1
Apache Beam version: 2.60.0


## 1. 샘플 데이터를 다운로드합니다.

[A Million News Headlines](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/SYBGZL#) 데이터세트에는 평판이 좋은 Australian Broadcasting Corp. (ABC)에서 공급한 15년치의 뉴스 헤드라인이 수록되어 있습니다. 이 뉴스 데이터세트에는 호주에 보다 세분화된 초점을 두고 2003년 초부터 2017년 말까지 전 세계적으로 일어난 주목할만한 사건에 대한 역사적 기록이 요약되어 있습니다.

**형식**: 탭으로 구분된 2열 데이터: 1) 발행일 및 2) 헤드라인 텍스트. 여기서는 헤드라인 텍스트에만 관심이 있습니다.


In [4]:
!wget 'https://dataverse.harvard.edu/api/access/datafile/3450625?format=tab&gbrecs=true' -O raw.tsv
!wc -l raw.tsv
!head raw.tsv

--2024-11-07 02:12:01--  https://dataverse.harvard.edu/api/access/datafile/3450625?format=tab&gbrecs=true
Resolving dataverse.harvard.edu (dataverse.harvard.edu)... 3.208.36.246, 3.223.176.15, 3.228.243.207
Connecting to dataverse.harvard.edu (dataverse.harvard.edu)|3.208.36.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57600231 (55M) [text/tab-separated-values]
Saving to: ‘raw.tsv’

raw.tsv             100%[===================>]  54.93M  48.1MB/s    in 1.1s    

2024-11-07 02:12:03 (48.1 MB/s) - ‘raw.tsv’ saved [57600231/57600231]

1103664 raw.tsv
publish_date	headline_text
20030219	"aba decides against community broadcasting licence"
20030219	"act fire witnesses must be aware of defamation"
20030219	"a g calls for infrastructure protection summit"
20030219	"air nz staff in aust strike for pay rise"
20030219	"air nz strike to affect australian travellers"
20030219	"ambitious olsson wins triple jump"
20030219	"antic delighted with record breaking barca"


단순화를 위해 헤드라인 텍스트만 유지하고 발행일은 제거합니다.

In [53]:
import pandas as pd
df = pd.read_csv('/content/sample_data/민원분류.csv')
df1=df['UpdatedTitle']
df1.to_csv('/content/sample_data/민원분류_title.csv',index=False)
df1.head()

,UpdatedTitle
0,수리요청바랍니다
1,안녕하세요 여기는. 오정동 우체국 옆 상가 주차장 입
2,쓰레기기를 낮애버려놓아 더니기불편해요* 안전신문고
3,표디판이 기울어져 있어요바르게 다시 설치요청* 안
4,주차 차단봉 설치


In [55]:
!rm -r corpus
!mkdir corpus

with open('corpus/text.txt', 'w') as out_file:
  with open('/content/sample_data/민원분류_title.csv', 'r') as in_file:
    for line in in_file:
      headline = line.split('\t')[0].strip().strip('"')
      out_file.write(headline+"\n")

In [56]:
!tail corpus/text.txt

수리 부탁드립니다 * 안전신문고 신고파일(사진·동영
브레이크등 미작동
수리 부탁드립니다 * 안전신문고 신고파일(사진·동영
가로등 수리 부탁드립니다 * 안전신문고 신고파일(사
가로등 수리 부탁드립니다 * 안전신문고 신고파일(사
가로등 수리 부탁드립니다 * 안전신문고 신고파일(사
가로등 수리 부탁드립니다 * 안전신문고 신고파일(사
브레이크등 미작동
수리 부탁 드리겠습니다* 안전신문고 신고파일(사진·
번호등 불량 제보합니다차량 소유주에게 안내 부탁드립니


## 2. 데이터에 대한 임베딩 생성하기

이 튜토리얼에서는 [Neural Network Language Model(NNLM)](https://tfhub.dev/google/nnlm-en-dim128/2)를 사용하여 헤드라인 데이터에 대한 임베딩을 생성합니다. 그런 다음 문장 임베딩을 사용하여 문장 수준의 의미 유사성을 쉽게 계산할 수 있습니다. Apache Beam을 사용하여 임베딩 생성 프로세스를 실행합니다.

### 임베딩 추출 메서드

In [57]:
embed_fn = None

def generate_embeddings(text, model_url, random_projection_matrix=None):
  # Beam will run this function in different processes that need to
  # import hub and load embed_fn (if not previously loaded)
  global embed_fn
  if embed_fn is None:
    embed_fn = hub.load(model_url)
  embedding = embed_fn(text).numpy()
  if random_projection_matrix is not None:
    embedding = embedding.dot(random_projection_matrix)
  return text, embedding


### tf.Example 메서드로 변환하기

In [58]:
def to_tf_example(entries):
  examples = []

  text_list, embedding_list = entries
  for i in range(len(text_list)):
    text = text_list[i]
    embedding = embedding_list[i]

    features = {
        'text': tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[text.encode('utf-8')])),
        'embedding': tf.train.Feature(
            float_list=tf.train.FloatList(value=embedding.tolist()))
    }

    example = tf.train.Example(
        features=tf.train.Features(
            feature=features)).SerializeToString(deterministic=True)

    examples.append(example)

  return examples

### Beam 파이프라인

In [59]:
def run_hub2emb(args):
  '''Runs the embedding generation pipeline'''

  options = beam.options.pipeline_options.PipelineOptions(**args)
  args = namedtuple("options", args.keys())(*args.values())

  with beam.Pipeline(args.runner, options=options) as pipeline:
    (
        pipeline
        | 'Read sentences from files' >> beam.io.ReadFromText(
            file_pattern=args.data_dir)
        | 'Batch elements' >> util.BatchElements(
            min_batch_size=args.batch_size, max_batch_size=args.batch_size)
        | 'Generate embeddings' >> beam.Map(
            generate_embeddings, args.model_url, args.random_projection_matrix)
        | 'Encode to tf example' >> beam.FlatMap(to_tf_example)
        | 'Write to TFRecords files' >> beam.io.WriteToTFRecord(
            file_path_prefix='{}/emb'.format(args.output_dir),
            file_name_suffix='.tfrecords')
    )

### 무작위 투영 가중치 행렬 생성하기

[무작위 투영](https://en.wikipedia.org/wiki/Random_projection)은 유클리드 공간에 있는 점 집합의 차원을 줄이는 데 사용되는 간단하지만 강력한 기술입니다. 이론적 배경은 [Johnson-Lindenstrauss 보조 정리](https://en.wikipedia.org/wiki/Johnson%E2%80%93Lindenstrauss_lemma)를 참조하세요.

무작위 투영으로 임베딩의 차원을 줄이면 ANN 인덱스를 빌드하고 쿼리하는 데 필요한 시간이 줄어듭니다.

이 튜토리얼에서는 [Scikit-learn](https://en.wikipedia.org/wiki/Random_projection#Gaussian_random_projection) 라이브러리의 [가우스 무작위 투영](https://scikit-learn.org/stable/modules/random_projection.html#gaussian-random-projection)을 사용합니다.

In [60]:
def generate_random_projection_weights(original_dim, projected_dim):
  random_projection_matrix = None
  random_projection_matrix = gaussian_random_matrix(
      n_components=projected_dim, n_features=original_dim).T
  print("A Gaussian random weight matrix was creates with shape of {}".format(random_projection_matrix.shape))
  print('Storing random projection matrix to disk...')
  with open('random_projection_matrix', 'wb') as handle:
    pickle.dump(random_projection_matrix,
                handle, protocol=pickle.HIGHEST_PROTOCOL)

  return random_projection_matrix

### 매개변수 설정하기

무작위 투영 없이 원래 임베딩 공간을 사용하여 인덱스를 빌드하려면 `projected_dim` 매개변수를 `None`으로 설정합니다. 그러면 높은 차원의 임베딩에 대한 인덱싱 스텝이 느려집니다.

In [61]:
def generate_random_projection_weights(original_dim, projected_dim):
    # Gaussian 랜덤 행렬 생성
    random_projection_matrix = gaussian_random_matrix(
        n_components=projected_dim, n_features=original_dim).T
    print("A Gaussian random weight matrix was created with shape of {}".format(random_projection_matrix.shape))
    return random_projection_matrix

In [62]:
model_url = 'https://tfhub.dev/google/nnlm-en-dim128/2' #@param {type:"string"}
projected_dim = 64  #@param {type:"number"}

### 파이프라인 실행하기

In [63]:
import tempfile
# Instead of importing gaussian_random_matrix, import GaussianRandomProjection
from sklearn.random_projection import GaussianRandomProjection
import numpy as np # Import numpy for array operations

output_dir = tempfile.mkdtemp()
original_dim = hub.load(model_url)(['']).shape[1]
random_projection_matrix = None

if projected_dim:
  # Create a GaussianRandomProjection object
  transformer = GaussianRandomProjection(n_components=projected_dim)
  # Fit the transformer to any data with the correct number of features (original_dim)
  # Here we use a dummy array for fitting
  transformer.fit(np.zeros((1, original_dim)))
  # Get the projection matrix
  random_projection_matrix = transformer.components_.T
  print("A Gaussian random weight matrix was created with shape of {}".format(random_projection_matrix.shape))


args = {
    'job_name': 'hub2emb-{}'.format(datetime.utcnow().strftime('%y%m%d-%H%M%S')),
    'runner': 'DirectRunner',
    'batch_size': 1024,
    'data_dir': 'corpus/*.txt',
    'output_dir': output_dir,
    'model_url': model_url,
    'random_projection_matrix': random_projection_matrix,
}

print("Pipeline args are set.")
args

A Gaussian random weight matrix was created with shape of (128, 64)
Pipeline args are set.


{'job_name': 'hub2emb-241107-031452',
 'runner': 'DirectRunner',
 'batch_size': 1024,
 'data_dir': 'corpus/*.txt',
 'output_dir': '/tmp/tmpex0u2i5l',
 'model_url': 'https://tfhub.dev/google/nnlm-en-dim128/2',
 'random_projection_matrix': array([[ 0.0018381 , -0.03032105, -0.18692303, ..., -0.02025734,
         -0.12891474, -0.0048825 ],
        [-0.14750635, -0.09745011, -0.03123811, ..., -0.18726707,
         -0.05538086,  0.0398117 ],
        [ 0.01147445, -0.12545985,  0.13579665, ...,  0.00341372,
          0.02885464,  0.01389387],
        ...,
        [ 0.1128564 , -0.19083163,  0.14494908, ..., -0.24827825,
         -0.14426842, -0.11296264],
        [ 0.12261419, -0.00303812, -0.12378974, ...,  0.24440566,
         -0.1132943 ,  0.15731792],
        [-0.06008186, -0.18666956,  0.08784254, ..., -0.0380415 ,
          0.01906254, -0.1402525 ]])}

In [64]:
print("Running pipeline...")
%time run_hub2emb(args)
print("Pipeline is done.")

        -0.12891474, -0.0048825 ],
       [-0.14750635, -0.09745011, -0.03123811, ..., -0.18726707,
        -0.05538086,  0.0398117 ],
       [ 0.01147445, -0.12545985,  0.13579665, ...,  0.00341372,
         0.02885464,  0.01389387],
       ...,
       [ 0.1128564 , -0.19083163,  0.14494908, ..., -0.24827825,
        -0.14426842, -0.11296264],
       [ 0.12261419, -0.00303812, -0.12378974, ...,  0.24440566,
        -0.1132943 ,  0.15731792],
       [-0.06008186, -0.18666956,  0.08784254, ..., -0.0380415 ,
         0.01906254, -0.1402525 ]])}


Running pipeline...
CPU times: user 2.43 s, sys: 1.44 s, total: 3.87 s
Wall time: 2.89 s
Pipeline is done.


In [65]:
!ls {output_dir}

emb-00000-of-00001.tfrecords


생성된 임베딩의 일부를 읽습니다.

In [66]:
embed_file = os.path.join(output_dir, 'emb-00000-of-00001.tfrecords')
sample = 5

# Create a description of the features.
feature_description = {
    'text': tf.io.FixedLenFeature([], tf.string),
    'embedding': tf.io.FixedLenFeature([projected_dim], tf.float32)
}

def _parse_example(example):
  # Parse the input `tf.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example, feature_description)

dataset = tf.data.TFRecordDataset(embed_file)
for record in dataset.take(sample).map(_parse_example):
  print("{}: {}".format(record['text'].numpy().decode('utf-8'), record['embedding'].numpy()[:10]))


UpdatedTitle: [-0.09777419  0.16038282  0.07757288  0.23370871  0.07652647 -0.12696837
 -0.07989868 -0.14378484 -0.17065716  0.13668518]
수리요청바랍니다: [ 0.01736344 -0.1256771  -0.02362035 -0.00696415 -0.08670127 -0.10139033
  0.15703374  0.00801026 -0.0426084   0.10337427]
안녕하세요 여기는. 오정동 우체국 옆 상가 주차장 입: [-0.0396103   0.26016995  0.1283808   0.00653004  0.05674693  0.16520491
 -0.05677558 -0.2530557   0.07675996  0.03215756]
쓰레기기를  낮애버려놓아 더니기불편해요* 안전신문고: [-0.05548347  0.07151826  0.11606419  0.20420715 -0.104642   -0.03690207
 -0.29866475 -0.09837242 -0.02509502 -0.05896591]
표디판이 기울어져 있어요바르게 다시 설치요청* 안: [ 0.05085089 -0.1527188   0.26022303  0.20919639 -0.20689072  0.1810183
  0.05357693 -0.26560214  0.1782473  -0.15379234]


## 3. 임베딩을 위한 ANN 인덱스 빌드하기

[ANNOY](https://github.com/spotify/annoy)(Approximate Nearest Neighbors Oh Yeah)는 주어진 쿼리 지점에 가까운 공간의 지점을 검색하기 위한 Python 바인딩이 있는 C++ 라이브러리입니다. 또한 메모리에 매핑되는 대규모 읽기 전용 파일 기반 데이터 구조를 생성합니다. 이는 음악 추천을 위해 [Spotify](https://www.spotify.com)에서 구축하고 사용합니다. 관심이 있으면 [NGT](https://github.com/yahoojapan/NGT), [FAISS](https://github.com/facebookresearch/faiss) 등과 같은 ANNOY의 다른 대안도 시도해볼 수 있습니다.

In [115]:
def build_index(embedding_files_pattern, index_filename, vector_length,
    metric='angular', num_trees=100):
  '''Builds an ANNOY index'''

  annoy_index = annoy.AnnoyIndex(vector_length, metric=metric)
  # Mapping between the item and its identifier in the index
  mapping = {}

  embed_files = tf.io.gfile.glob(embedding_files_pattern)
  num_files = len(embed_files)
  print('Found {} embedding file(s).'.format(num_files))

  item_counter = 0
  for i, embed_file in enumerate(embed_files):
    print('Loading embeddings in file {} of {}...'.format(i+1, num_files))
    dataset = tf.data.TFRecordDataset(embed_file)
    for record in dataset.map(_parse_example):
      text = record['text'].numpy().decode("utf-8")
      embedding = record['embedding'].numpy()
      mapping[item_counter] = text
      annoy_index.add_item(item_counter, embedding)
      item_counter += 1
      if item_counter % 100000 == 0:
        print('{} items loaded to the index'.format(item_counter))

  print('A total of {} items added to the index'.format(item_counter))

  print('Building the index with {} trees...'.format(num_trees))
  annoy_index.build(n_trees=num_trees)
  print('Index is successfully built.')

  print('Saving index to disk...')
  annoy_index.save(index_filename)
  print('Index is saved to disk.')
  print("Index file size: {} GB".format(
    round(os.path.getsize(index_filename) / float(1024 ** 3), 2)))
  annoy_index.unload()

  print('Saving mapping to disk...')
  with open(index_filename + '.mapping', 'wb') as handle:
    pickle.dump(mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print('Mapping is saved to disk.')
  print("Mapping file size: {} MB".format(
    round(os.path.getsize(index_filename + '.mapping') / float(1024 ** 2), 2)))

In [116]:
embedding_files = "{}/emb-*.tfrecords".format(output_dir)
embedding_dimension = projected_dim
index_filename = "index"

!rm {index_filename}
!rm {index_filename}.mapping

%time build_index(embedding_files, index_filename, embedding_dimension)

Found 1 embedding file(s).
Loading embeddings in file 1 of 1...
A total of 10374 items added to the index
Building the index with 100 trees...
Index is successfully built.
Saving index to disk...
Index is saved to disk.
Index file size: 0.02 GB
Saving mapping to disk...
Mapping is saved to disk.
Mapping file size: 0.72 MB
CPU times: user 7.79 s, sys: 317 ms, total: 8.11 s
Wall time: 6.4 s


In [117]:
!ls

corpus	index  index.ann  index.mapping  raw.tsv  sample_data


## 4. 유사성 일치에 인덱스 사용하기

이제 ANN 인덱스를 사용하여 의미상 입력 쿼리에 가까운 뉴스 헤드라인을 찾을 수 있습니다.

### 인덱스 및 매핑 파일 로드하기

In [118]:
index = annoy.AnnoyIndex(embedding_dimension)
index.load(index_filename, prefault=True)
print('Annoy index is loaded.')
with open(index_filename + '.mapping', 'rb') as handle:
  mapping = pickle.load(handle)
print('Mapping file is loaded.')


Annoy index is loaded.
Mapping file is loaded.


<ipython-input-118-4bcb1204a779>:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  index = annoy.AnnoyIndex(embedding_dimension)


### 유사성 일치 메서드

In [119]:
def find_similar_items(embedding, num_matches=5):
  '''Finds similar items to a given embedding in the ANN index'''
  ids = index.get_nns_by_vector(
  embedding, num_matches, search_k=-1, include_distances=False)
  items = [mapping[i] for i in ids]
  return items

### 주어진 쿼리에서 임베딩 추출하기

In [120]:
# Load the TF-Hub model
print("Loading the TF-Hub model...")
%time embed_fn = hub.load(model_url)
print("TF-Hub model is loaded.")

random_projection_matrix = None
if os.path.exists('random_projection_matrix'):
  print("Loading random projection matrix...")
  with open('random_projection_matrix', 'rb') as handle:
    random_projection_matrix = pickle.load(handle)
  print('random projection matrix is loaded.')

def extract_embeddings(query):
  '''Generates the embedding for the query'''
  query_embedding =  embed_fn([query])[0].numpy()
  if random_projection_matrix is not None:
    query_embedding = query_embedding.dot(random_projection_matrix)
  return query_embedding


Loading the TF-Hub model...
CPU times: user 624 ms, sys: 679 ms, total: 1.3 s
Wall time: 1.31 s
TF-Hub model is loaded.


In [121]:
extract_embeddings("Hello Machine Learning!")[:10]

array([-0.13498034, -0.1588092 ,  0.10572661, -0.00282711,  0.02178992,
       -0.01289755, -0.04587182,  0.10151501, -0.02941302, -0.14763536],
      dtype=float32)

In [122]:
import os
import pickle

def find_similar_items(embedding, num_matches=5):
    '''Finds similar items to a given embedding in the ANN index'''
    # Check if the embedding dimension matches the index's expected dimension
    if embedding.shape[0] != index.d:
        # If not, try to project the embedding to the correct dimension
        if random_projection_matrix is not None and embedding.shape[0] == random_projection_matrix.shape[0]:
            embedding = embedding.dot(random_projection_matrix)
        else:
            raise ValueError(f"Embedding dimension ({embedding.shape[0]}) does not match index dimension ({index.d})."
                             f" Please ensure that the embedding model and random projection matrix are consistent with the index.")

    ids = index.get_nns_by_vector(
        embedding, num_matches, search_k=-1, include_distances=False)
    items = [mapping[i] for i in ids]
    return items

In [123]:
# Load the TF-Hub model
print("Loading the TF-Hub model...")
# %time embed_fn = hub.load(model_url) # Assumed to be loaded elsewhere
print("TF-Hub model is loaded.")

random_projection_matrix = None
if os.path.exists('random_projection_matrix'):
  print("Loading random projection matrix...")
  with open('random_projection_matrix', 'rb') as handle:
    random_projection_matrix = pickle.load(handle)
  print('random projection matrix is loaded.')

def extract_embeddings(query):
  '''Generates the embedding for the query'''
  global random_projection_matrix # Declare random_projection_matrix as global
  query_embedding =  embed_fn([query])[0].numpy()
  if random_projection_matrix is not None:
    # Check if the shapes are compatible for dot product
    if query_embedding.shape[0] != random_projection_matrix.shape[0]:
      # If the shapes are not compatible, and random_projection_matrix is
      # intended for dimensionality reduction, then likely
      # random_projection_matrix needs to have dimensions (128, X)
      # where X is the target dimensionality.
      # Instead of transposing, reshape or recreate random_projection_matrix:
      random_projection_matrix = np.random.rand(query_embedding.shape[0], 512)
      # Replace 512 with your desired target dimensionality if different.
    query_embedding = query_embedding.dot(random_projection_matrix)
  return query_embedding

Loading the TF-Hub model...
TF-Hub model is loaded.


In [124]:
import os
import pickle
from annoy import AnnoyIndex # Importing AnnoyIndex

def find_similar_items(embedding, num_matches=5):
    '''Finds similar items to a given embedding in the ANN index'''
    # Check if the embedding dimension matches the index's expected dimension
    # Use index.f to get the dimension instead of index.d
    if embedding.shape[0] != index.f:
        # If not, try to project the embedding to the correct dimension
        if random_projection_matrix is not None and embedding.shape[0] == random_projection_matrix.shape[0]:
            embedding = embedding.dot(random_projection_matrix)
        else:
            raise ValueError(f"Embedding dimension ({embedding.shape[0]}) does not match index dimension ({index.f})."
                             f" Please ensure that the embedding model and random projection matrix are consistent with the index.")

    ids = index.get_nns_by_vector(
        embedding, num_matches, search_k=-1, include_distances=False)
    items = [mapping[i] for i in ids]
    return items

### 가장 유사한 항목을 찾기 위한 쿼리 입력하기

In [150]:
# ipython-input-35-7cbfba78879e

#@title { run: "auto" }
query = "무단방치차량 치워주세요" #@param {type:"string"}

# Assuming these variables are defined or initialized elsewhere in your code
# REPLACE ... WITH A VALID ANN INDEX OBJECT
# For example, if you're using Annoy:
!pip install annoy
from annoy import AnnoyIndex
# Assume embedding_dimension is defined
embedding_dimension = 128  # Replace with your actual embedding dimension
index = AnnoyIndex(embedding_dimension, 'angular')
index.load('index.ann') # Load a pre-built index - ensure the file exists

# REPLACE ... WITH THE ACTUAL MAPPING
#mapping = {}

# REPLACE ... WITH THE ACTUAL RANDOM PROJECTION MATRIX (IF USED)
import numpy as np
random_projection_matrix = None
if os.path.exists('random_projection_matrix'):
    with open('index.mapping', 'rb') as handle:
        random_projection_matrix = pickle.load(handle)
    print("Random projection matrix loaded successfully.")

print("Generating embedding for the query...")
%time query_embedding = extract_embeddings(query)

print("")
print("Finding relevant items in the index...")
%time items = find_similar_items(query_embedding, 10)

print("")
print("Results:")
print("=========")
for item in items:
  print(item)

Generating embedding for the query...
CPU times: user 3.47 ms, sys: 0 ns, total: 3.47 ms
Wall time: 2.93 ms

Finding relevant items in the index...
CPU times: user 1.28 ms, sys: 784 µs, total: 2.07 ms
Wall time: 1.56 ms

Results:
Unknown item
Unknown item
Unknown item
Unknown item
Unknown item
Unknown item
Unknown item
Unknown item
Unknown item
Unknown item


In [134]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
from annoy import AnnoyIndex
import pickle

# 1. TF-Hub 모델 로드
model_url = "https://tfhub.dev/google/nnlm-en-dim128/2"  # 사용할 TF-Hub 모델 URL (128 차원 임베딩)
embed_fn = hub.load(model_url)

# 2. 파일 로드 및 데이터 준비
file_path = '/content/sample_data/민원분류_title.csv'
data_df = pd.read_csv(file_path)
data = data_df['UpdatedTitle'].dropna().tolist()  # 필요한 열로 수정

# 설정
embedding_dimension = 128  # TF-Hub 모델에 따른 임베딩 차원
index = AnnoyIndex(embedding_dimension, 'angular')

# 매핑 딕셔너리 초기화
mapping = {}

# 3. 임베딩 생성 함수
def extract_embeddings(text):
    embedding = embed_fn([text])[0].numpy()  # TF-Hub 모델로 임베딩 생성
    return embedding

# 4. 인덱스에 데이터 추가
for i, text in enumerate(data):
    embedding = extract_embeddings(text)
    index.add_item(i, embedding)  # Annoy 인덱스에 임베딩 추가
    mapping[i] = text  # 매핑에 텍스트 저장

# 5. 인덱스 빌드 및 저장
index.build(10)  # 트리 개수 조정 가능
index.save('index.ann')

# 6. 매핑 파일 저장
with open('index.mapping', 'wb') as handle:
    pickle.dump(mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("ANN index and mapping files saved successfully.")

# 7. 유사 항목 검색 함수
def find_similar_items(query_embedding, num_matches=5):
    item_ids = index.get_nns_by_vector(query_embedding, num_matches)
    items = [mapping.get(item_id, "Unknown item") for item_id in item_ids]
    return items

# 8. 쿼리 입력 및 임베딩 생성 후 검색
query = "버스차로 위반"  # 예시 질의어
query_embedding = extract_embeddings(query)  # 쿼리에 대한 임베딩 생성

# 검색 결과 출력
items = find_similar_items(query_embedding, 5)

print("\nResults for query '{}':".format(query))
print("=========")
for item in items:
    print(item)

ANN index and mapping files saved successfully.

Results for query '버스차로 위반':
 자전거보관소 직사각형 개선 하세요
금연구역  개선 하세요
아파트 헬스장 관련 문의
기타 불법 주정차 신고입니다. 코너 꺾는곳에다가 차를 
아파트 입구 주차봉 설치 관련


## 더 자세히 알고 싶나요?

[tensorflow.org](https://www.tensorflow.org/)에서 TensorFlow에 대해 자세히 알아보고 [tensorflow.org/hub](https://www.tensorflow.org/hub/)에서 TF-Hub API 설명서를 확인할 수 있습니다. 추가적인 텍스트 임베딩 모듈 및 이미지 특성 벡터 모듈을 포함해 [tfhub.dev](https://tfhub.dev/)에서 사용 가능한 TensorFlow Hub 모델을 찾아보세요.

빠르게 진행되는 Google의 머신러닝 실무 개요 과정인 [머신러닝 집중 과정](https://developers.google.com/machine-learning/crash-course/)도 확인해 보세요.

In [127]:
import pickle
from annoy import AnnoyIndex

# 인덱스 로드
embedding_dimension = 512  # 실제 임베딩 차원에 맞게 설정
index = AnnoyIndex(embedding_dimension, 'angular')
index.load('index.ann')  # 저장된 인덱스 파일 로드

# `index.mapping` 파일을 pickle로 로드
with open('index.mapping', 'rb') as handle:
    mapping = pickle.load(handle)
print("Mapping loaded successfully.")

# 유사 항목 검색 함수
def find_similar_items(embedding, num_matches=5):
    ids = index.get_nns_by_vector(embedding, num_matches, search_k=-1, include_distances=False)
    items = [mapping.get(i, "Unknown item") for i in ids]
    return items

# 예시 쿼리
query = "쓰레기 무단투기"
query_embedding = extract_embeddings(query)  # 쿼리에 대한 임베딩 생성

# 검색 결과 출력
items = find_similar_items(query_embedding, 10)
print("\nResults:")
print("=========")
for item in items:
    print(item)

Mapping loaded successfully.

Results:
수리요청바랍니다
UpdatedTitle
안녕하세요 여기는. 오정동 우체국 옆 상가 주차장 입


In [110]:
import pickle
from annoy import AnnoyIndex
import tensorflow as tf
import numpy as np

# 예시 데이터 및 인덱스 설정
embedding_dimension = 512
index = AnnoyIndex(embedding_dimension, 'angular')

# 새로 생성한 `mapping`을 저장하는 코드
mapping = {0: "Example item 1", 1: "Example item 2", 2: "Example item 3"}  # 실제 데이터로 대체 필요

# Annoy 인덱스 빌드 및 저장 (임베딩은 예제용, 실제 임베딩으로 대체)
index.add_item(0, np.random.rand(embedding_dimension))
index.add_item(1, np.random.rand(embedding_dimension))
index.add_item(2, np.random.rand(embedding_dimension))
index.build(10)
index.save('index.ann')

# 새로 생성한 mapping 저장
with open('index.mapping', 'wb') as handle:
    pickle.dump(mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("Mapping file saved successfully.")

Mapping file saved successfully.


In [111]:
# `index.mapping` 파일을 pickle로 로드
with open('index.mapping', 'rb') as handle:
    loaded_mapping = pickle.load(handle)
print("Loaded mapping:", loaded_mapping)

Loaded mapping: {0: 'Example item 1', 1: 'Example item 2', 2: 'Example item 3'}


In [100]:
from annoy import AnnoyIndex
import numpy as np

# 설정
embedding_dimension = 512
index = AnnoyIndex(embedding_dimension, 'angular')

# 예시 데이터 추가 및 인덱스 빌드
for i in range(1000):  # 예시 데이터의 수에 맞게 반복
    vector = np.random.rand(embedding_dimension)
    index.add_item(i, vector)

index.build(1000)  # 인덱스 빌드
index.save('index.ann')

# 매핑 데이터 확인
mapping = {i: f"Example item {i}" for i in range(10)}

# 예시 embedding 생성 함수
def extract_embeddings(text):
    return np.random.rand(embedding_dimension)

# 예시 유사 항목 검색 함수
def find_similar_items(query_embedding, num_items=5):
    item_ids = index.get_nns_by_vector(query_embedding, num_items)
    items = [mapping.get(item_id, "Unknown item") for item_id in item_ids]
    return items

# 검색 쿼리 임베딩 생성 및 유사 항목 찾기
query = "쓰레기무단투기"
query_embedding = extract_embeddings(query)
items = find_similar_items(query_embedding, 5)

# 결과 출력
print("\nResults:")
print("=========")
for item in items:
    print(item)


Results:
Example item 3
Example item 6
Example item 8
Example item 5
Example item 2


In [95]:
index = AnnoyIndex(embedding_dimension, 'angular')
index.load('index.ann')  # 저장한 파일 로드

True